In [ ]:
import os
import sys
import path
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
sys.path.insert(0, path.Path(os.getcwd()).joinpath("..").abspath())

### Image

In [ ]:
import os

import torch

from catalyst import utils
from catalyst.data import transforms
from catalyst.contrib.datasets import CIFAR10

from captum.attr import visualization as viz
from captum.attr import (
    Saliency, 
    IntegratedGradients,
    NoiseTunnel,
    LayerGradCam, 
    FeatureAblation, 
    LayerActivation, 
    LayerAttribution
)

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def unnormalize(tensor: torch.Tensor, mean, std, inplace=False):
    """Normalize a tensor image with mean and standard deviation.

    .. note::
        This transform acts out of place by default, i.e., it does not mutates the input tensor.

    Args:
        tensor: Tensor image of size (C, H, W) to be normalized.
        mean: Sequence of means for each channel.
        std: Sequence of standard deviations for each channel.
        inplace(bool,optional): Bool to make this operation inplace.

    Returns:
        torch.Tensor: Normalized Tensor image.

    Raises:
        TypeError: if `tensor` is not torch.Tensor
    """
    if not (torch.is_tensor(tensor) and tensor.ndimension() == 3):
        raise TypeError("tensor is not a torch image.")

    if not inplace:
        tensor = tensor.clone()

    dtype = tensor.dtype
    mean = torch.as_tensor(mean, dtype=dtype, device=tensor.device)
    std = torch.as_tensor(std, dtype=dtype, device=tensor.device)
    tensor.mul_(std[:, None, None]).add_(mean[:, None, None])
    return tensor

transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
valid_dataset = CIFAR10(os.getcwd(), train=False, download=True, transform=transform)

In [ ]:
features, label = valid_dataset[10]

features = torch.tensor(features).unsqueeze(0)
features.requires_grad = True

label = torch.tensor(label).unsqueeze(0)

In [ ]:
plt.imshow(
    unnormalize(
        features[0].detach(), 
        (0.5, 0.5, 0.5), 
        (0.5, 0.5, 0.5)
    ).permute(1, 2, 0)
)

In [ ]:
from image.train import resnet9


class ModelWrapper(torch.nn.Module):
    def __init__(self, module):
        super().__init__()
        self.module = module
    def forward(self, x):
        _, y = self.module(x)
        return y

logdir = "../logs/.../"
checkpoint = utils.load_checkpoint(f"{logdir}/checkpoints/best.pth")
model = resnet9(in_channels=3, num_classes=10)
utils.unpack_checkpoint(checkpoint, model=model)
model = ModelWrapper(model)

In [ ]:
def attribute_image_features(algorithm, model, features, label, **kwargs):
    model.zero_grad()
    tensor_attributions = algorithm.attribute(
        features,
        target=label, 
        **kwargs
    )
    
    return tensor_attributions

In [ ]:
saliency = Saliency(model)
grads = saliency.attribute(features, target=label.item())
# grads = np.transpose(grads.squeeze().cpu().detach().numpy(), (1, 2, 0))

In [ ]:
ig = IntegratedGradients(model)
attr_ig, delta = attribute_image_features(
    ig, 
    model,
    features, 
    label,
    baselines=features * 0, 
    return_convergence_delta=True
)
# attr_ig = np.transpose(attr_ig.squeeze().cpu().detach().numpy(), (1, 2, 0))
print('Approximation delta: ', abs(delta))

In [ ]:
ig = IntegratedGradients(model)
nt = NoiseTunnel(ig)
attr_ig_nt = attribute_image_features(
    nt, 
    model,
    features, 
    label,
    baselines=features * 0, 
    nt_type='smoothgrad_sq',
    nt_samples=100, 
    stdevs=0.2
)

In [ ]:
plt.imshow(
    unnormalize(
        features[0].detach(), 
        (0.5, 0.5, 0.5), 
        (0.5, 0.5, 0.5)
    ).permute(1, 2, 0)
)

In [ ]:
plt.matshow(grads.permute(0, 2, 3, 1).detach().cpu().numpy()[0])

In [ ]:
plt.matshow(attr_ig.permute(0, 2, 3, 1).detach().cpu().numpy()[0])

In [ ]:
plt.matshow(attr_ig_nt.permute(0, 2, 3, 1).detach().cpu().numpy()[0])

### Video